In [ ]:
class Init_Boot:

    def __init__(self):
        #パス関連変数の生成
        self.base_dir = base_dir
        self.data_dir = os.path.join(self.base_dir,data_dir)
        self.save_dir = os.path.join(self.data_dir,save_dir)
        self.model_dir = os.path.join(self.base_dir,model_dir)
        self.log_dir = os.path.join(self.base_dir,log_dir)

        #生成するファイル名変数の生成
        self.file_train = os.path.join(
            self.data_dir,
            str(os.path.splitext(os.path.basename(__file__))[0]
            + "_" + file_train + ".npz")
        )
        self.file_test = os.path.join(
            self.data_dir,
            str(os.path.splitext(os.path.basename(__file__))[0]
            + "_" + file_test + ".npz")
        )
        self.file_model_pca = os.path.join(
            self.model_dir,
            str(os.path.splitext(os.path.basename(__file__))[0]
            + "_" + file_model_pca + ".dat")
        )

        self.file_model_skAE = os.path.join(
            self.model_dir,
            str(os.path.splitext(os.path.basename(__file__))[0]
            + "_" + file_model_skAE + ".dat")
        )

        #他の変数の初期化
        self.br = br
        self.sr = sr
        self.r_wait = r_wait
        self.r_len = r_len
        self.a_idx = a_idx
        self.disp_spg = disp_spg
        self.axis_freq = axis_freq
        self.axis_tme = axis_tme
        self.aug_amt = aug_amt
        self.exp_ver = exp_ver

        #必要なインスタンスの生成
        self.pa = pyaudio.PyAudio()
        self.cfg = configparser.ConfigParser()

    """フォルダの生成"""
    def  elem_gen_folder(self,target_dir):
        if os.path.exists(target_dir):
            pass
        else:
            os.mkdir(target_dir)
            print("Created a directory:{0}".format(target_dir))

    """設定値のセット"""
    def elem_set_ini(self):
        x = self.cfg
        x["General"] = {
            "base_dir" : self.base_dir,
            "data_dir" : self.data_dir,
            "save_dir" : self.save_dir,
            "model_dir" : self.model_dir,
            "log_dir" : self.log_dir,
            "file_train" : self.file_train,
            "file_test" : self.file_test,
            "file_model_pca" : self.file_model_pca,
            "file_model_skAE" : self.file_model_skAE
        }

        x["Rec_param"] = {
            "br" : self.br,
            "sr" : self.sr,
            "r_wait" : self.r_wait,
            "r_len" : self.r_len,
            "disp_spg" : self.disp_spg
        }

        if self.a_idx == None:
            x["Rec_param"]["a_idx"] = ""
        else:
            x["Rec_param"]["a_idx"] = str(self.a_idx)

        x["DS_param"] = {
            "aug_amt" : self.aug_amt
        }

        if self.axis_freq == None:
            x["DS_param"]["axis_freq"] = ""
        else:
            x["DS_param"]["axis_freq"] = self.axis_freq.tolist()

        if self.axis_tme == None:
            x["DS_param"]["axis_tme"] = ""
        else:
            x["DS_param"]["axis_tme"] = self.axis_tme.tolist()

        x["PCA_param"] = {
            "exp_ver" : self.exp_ver
        }

        return x

    """iniファイルへの保存"""
    def elem_save_ini(self,cfg):
        with open(
            os.path.splitext(
                os.path.basename(__file__)
            )[0] + ".ini","w") as cfgfile:
            cfg.write(cfgfile)
        print ("Saved setting parameters.")

    """iniファイルのロード"""
    def elem_load_ini(self):
        x = self.cfg
        x.read(
            os.path.join(
                "./",str(
                    os.path.splitext(
                        os.path.basename(__file__)
                    )[0] + ".ini"
                )
            )
        )
        return x

    """設定値の読み出し"""
    def elem_get_ini(self,cfg):
        x = cfg

        #読み出し
        self.base_dir = x.get("General","base_dir")
        self.data_dir = x.get("General","data_dir")
        self.save_dir = x.get("General","save_dir")
        self.model_dir = x.get("General","model_dir")
        self.log_dir = x.get("General","log_dir")
        self.file_train = x.get("General","file_train")
        self.file_test = x.get("General","file_test")
        self.file_model_pca = x.get("General","file_model_pca")
        self.file_model_skAE = x.get("General","file_model_skAE")

        self.br = x.getint("Rec_param","br")
        self.sr = x.getint("Rec_param","sr")
        self.r_wait = x.getint("Rec_param","r_wait")
        self.r_len = x.getint("Rec_param","r_len")
        self.disp_spg = x.getboolean("Rec_param","disp_spg")

        self.a_idx = x.get("Rec_param","a_idx")
        if self.a_idx == "":
            self.a_idx = None
        else:
            self.a_idx = int(self.a_idx)

        self.axis_freq = x.get("DS_param","axis_freq")
        if self.axis_freq == "":
            self.axis_freq = None
        else:
            self.axis_freq = np.array(json.loads(self.axis_freq))

        self.axis_tme = x.get("DS_param","axis_tme")
        if self.axis_tme == "":
            self.axis_tme = None
        else:
            self.axis_tme = np.array(json.loads(self.axis_tme))
        
        self.aug_amt = x.getint("DS_param","aug_amt")
        self.exp_ver = x.getfloat("PCA_param","exp_ver")

    """オーディオインデックスの定義"""
    def elem_a_idx(self):
        print("***List of available audio devices:***")
        for i in range(self.pa.get_device_count()):
            print(i,self.pa.get_device_info_by_index(i).get("name"),sep = " - ")
        x = int(input("Select Audio device Index No."))
        print("***Selected audio device #{0}.***".format(x))
        return x


    """起動処理"""
    def proc_boot(self):
        #iniファイルの有無判定
        if os.path.exists(
            os.path.join("./",str(os.path.splitext(
                os.path.basename(__file__)
                )[0] + ".ini")
            )
        ):
            #iniファイルが存在していればロードする
            cfg = self.elem_load_ini()
            self.elem_get_ini(cfg)
            print("Loaded initial settings.\n***Audio device #{0} - {1} is selected***"\
            .format(self.a_idx,self.pa.get_device_info_by_index(self.a_idx)\
                .get("name")))
            #読み出したbase_dirとグローバル変数のbase_dirが異なる場合パス関連を上書き
            if self.base_dir != base_dir:
                print("Current directory was changed. -> Overwrite config.")
                del cfg #明示的に消しておく
                self.base_dir = base_dir,
                self.data_dir = os.path.join(self.base_dir,data_dir)
                self.save_dir = os.path.join(self.data_dir,save_dir)
                self.model_dir = os.path.join(self.base_dir,model_dir)
                self.log_dir = os.path.join(self.base_dir,log_dir)
                self.file_train = os.path.join(
                    self.data_dir,
                    str(os.path.splitext(os.path.basename(__file__))[0]
                    + file_train + ".npz")
                )
                self.file_test = os.path.join(
                    self.data_dir,
                    str(os.path.splitext(os.path.basename(__file__))[0]
                    + file_test + ".npz")
                )
                self.file_model_pca = os.path.join(
                    self.model_dir,
                    str(os.path.splitext(os.path.basename(__file__))[0]
                    + file_model_pca + ".dat")
                )
                self.file_model_skAE = os.path.join(
                    self.model_dir,
                    str(os.path.splitext(os.path.basename(__file__))[0]
                    + file_model_skAE + ".dat")
                )
                cfg = self.elem_set_ini()
                self.elem_save_ini(cfg)
            else:
                pass

        #iniファイルが存在しない場合、オーディオインデックスを定義してセーブ
        else:
            self.a_idx = self.elem_a_idx()
            cfg = self.elem_set_ini()
            self.elem_save_ini(cfg)
        
        #データフォルダ関連の生成
        self.elem_gen_folder(self.data_dir)
        self.elem_gen_folder(self.save_dir)
        self.elem_gen_folder(os.path.join(self.save_dir,"test"))
        self.elem_gen_folder(os.path.join(self.save_dir,"test","ok"))
        self.elem_gen_folder(os.path.join(self.save_dir,"test","ng"))
        self.elem_gen_folder(os.path.join(self.save_dir,"test","valid"))
        self.elem_gen_folder(self.model_dir)
        self.elem_gen_folder(self.log_dir)

        return self.base_dir,self.data_dir,self.save_dir,self.model_dir,\
            self.log_dir,self.file_train,self.file_test,self.file_model_pca,\
            self.file_model_skAE,self.br,self.sr,self.r_wait,self.r_len,self.a_idx,\
            self.disp_spg,self.axis_freq,self.axis_tme,self.aug_amt,self.exp_ver